In [4]:
# import pyforest
import pandas as pd
pd.set_option("display.max_columns", 200)
pd.set_option("display.float_format", lambda x: "%.2f" % x)
# pd.set_option('display.max_colwidth', 50)

import warnings

warnings.filterwarnings("ignore")

In [6]:
import spacy
import pandas as pd

In [19]:
df = pd.read_csv("data/flipkart_com-ecommerce_sample_1050.csv")
df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,55b85ea15a1536d46b7190ad6fff8ce7,2016-04-30 03:22:56 +0000,http://www.flipkart.com/elegance-polyester-mul...,Elegance Polyester Multicolor Abstract Eyelet ...,"[""Home Furnishing >> Curtains & Accessories >>...",CRNEG7BKMFFYHQ8Z,1899.0,899.0,55b85ea15a1536d46b7190ad6fff8ce7.jpg,False,Key Features of Elegance Polyester Multicolor ...,No rating available,No rating available,Elegance,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
1,7b72c92c2f6c40268628ec5f14c6d590,2016-04-30 03:22:56 +0000,http://www.flipkart.com/sathiyas-cotton-bath-t...,Sathiyas Cotton Bath Towel,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEGFZHGBXPHZUH,600.0,449.0,7b72c92c2f6c40268628ec5f14c6d590.jpg,False,Specifications of Sathiyas Cotton Bath Towel (...,No rating available,No rating available,Sathiyas,"{""product_specification""=>[{""key""=>""Machine Wa..."
2,64d5d4a258243731dc7bbb1eef49ad74,2016-04-30 03:22:56 +0000,http://www.flipkart.com/eurospa-cotton-terry-f...,Eurospa Cotton Terry Face Towel Set,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEG6SHXTDB2A2Y,NaN,NaN,64d5d4a258243731dc7bbb1eef49ad74.jpg,False,Key Features of Eurospa Cotton Terry Face Towe...,No rating available,No rating available,Eurospa,"{""product_specification""=>[{""key""=>""Material"",..."
3,d4684dcdc759dd9cdf41504698d737d8,2016-06-20 08:49:52 +0000,http://www.flipkart.com/santosh-royal-fashion-...,SANTOSH ROYAL FASHION Cotton Printed King size...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJT9UQWHDUBH4,2699.0,1299.0,d4684dcdc759dd9cdf41504698d737d8.jpg,False,Key Features of SANTOSH ROYAL FASHION Cotton P...,No rating available,No rating available,SANTOSH ROYAL FASHION,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
4,6325b6870c54cd47be6ebfbffa620ec7,2016-06-20 08:49:52 +0000,http://www.flipkart.com/jaipur-print-cotton-fl...,Jaipur Print Cotton Floral King sized Double B...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJTHNGWVGWWQU,2599.0,698.0,6325b6870c54cd47be6ebfbffa620ec7.jpg,False,Key Features of Jaipur Print Cotton Floral Kin...,No rating available,No rating available,Jaipur Print,"{""product_specification""=>[{""key""=>""Machine Wa..."


In [20]:
df_nlp = df[["uniq_id", "product_name", "product_category_tree", "description", "product_specifications"]]
df_nlp.head()

,uniq_id,product_name,product_category_tree,description,product_specifications
0,55b85ea15a1536d46b7190ad6fff8ce7,Elegance Polyester Multicolor Abstract Eyelet ...,"[""Home Furnishing >> Curtains & Accessories >>...",Key Features of Elegance Polyester Multicolor ...,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
1,7b72c92c2f6c40268628ec5f14c6d590,Sathiyas Cotton Bath Towel,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",Specifications of Sathiyas Cotton Bath Towel (...,"{""product_specification""=>[{""key""=>""Machine Wa..."
2,64d5d4a258243731dc7bbb1eef49ad74,Eurospa Cotton Terry Face Towel Set,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",Key Features of Eurospa Cotton Terry Face Towe...,"{""product_specification""=>[{""key""=>""Material"",..."
3,d4684dcdc759dd9cdf41504698d737d8,SANTOSH ROYAL FASHION Cotton Printed King size...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",Key Features of SANTOSH ROYAL FASHION Cotton P...,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
4,6325b6870c54cd47be6ebfbffa620ec7,Jaipur Print Cotton Floral King sized Double B...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",Key Features of Jaipur Print Cotton Floral Kin...,"{""product_specification""=>[{""key""=>""Machine Wa..."


In [21]:
df_nlp["product_category_tree"] = df_nlp["product_category_tree"].str.replace('\["', "").str.replace('\]"', "")
df_nlp.head()

C:\Users\matgi\AppData\Local\Temp\ipykernel_17764\298991014.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_nlp["product_category_tree"] = df_nlp["product_category_tree"].str.replace('\["', "").str.replace('\]"', "")
C:\Users\matgi\AppData\Local\Temp\ipykernel_17764\298991014.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp["product_category_tree"] = df_nlp["product_category_tree"].str.replace('\["', "").str.replace('\]"', "")


,uniq_id,product_name,product_category_tree,description,product_specifications
0,55b85ea15a1536d46b7190ad6fff8ce7,Elegance Polyester Multicolor Abstract Eyelet ...,Home Furnishing >> Curtains & Accessories >> C...,Key Features of Elegance Polyester Multicolor ...,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
1,7b72c92c2f6c40268628ec5f14c6d590,Sathiyas Cotton Bath Towel,Baby Care >> Baby Bath & Skin >> Baby Bath Tow...,Specifications of Sathiyas Cotton Bath Towel (...,"{""product_specification""=>[{""key""=>""Machine Wa..."
2,64d5d4a258243731dc7bbb1eef49ad74,Eurospa Cotton Terry Face Towel Set,Baby Care >> Baby Bath & Skin >> Baby Bath Tow...,Key Features of Eurospa Cotton Terry Face Towe...,"{""product_specification""=>[{""key""=>""Material"",..."
3,d4684dcdc759dd9cdf41504698d737d8,SANTOSH ROYAL FASHION Cotton Printed King size...,Home Furnishing >> Bed Linen >> Bedsheets >> S...,Key Features of SANTOSH ROYAL FASHION Cotton P...,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
4,6325b6870c54cd47be6ebfbffa620ec7,Jaipur Print Cotton Floral King sized Double B...,Home Furnishing >> Bed Linen >> Bedsheets >> J...,Key Features of Jaipur Print Cotton Floral Kin...,"{""product_specification""=>[{""key""=>""Machine Wa..."


In [22]:
df_nlp[["cat1", "cat2", "cat3", "cat4", "cat5", "cat6", "cat7"]] = df_nlp["product_category_tree"].str.split(">>", expand=True)

C:\Users\matgi\AppData\Local\Temp\ipykernel_17764\4166550304.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp[["cat1", "cat2", "cat3", "cat4", "cat5", "cat6", "cat7"]] = df_nlp["product_category_tree"].str.split(">>", expand=True)
C:\Users\matgi\AppData\Local\Temp\ipykernel_17764\4166550304.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp[["cat1", "cat2", "cat3", "cat4", "cat5", "cat6", "cat7"]] = df_nlp["product_category_tree"].str.split(">>", expand=True)
C:\Users\matgi\AppData\Local\T

In [23]:
df_nlp.head(2)

,uniq_id,product_name,product_category_tree,description,product_specifications,cat1,cat2,cat3,cat4,cat5,cat6,cat7
0,55b85ea15a1536d46b7190ad6fff8ce7,Elegance Polyester Multicolor Abstract Eyelet ...,Home Furnishing >> Curtains & Accessories >> C...,Key Features of Elegance Polyester Multicolor ...,"{""product_specification""=>[{""key""=>""Brand"", ""v...",Home Furnishing,Curtains & Accessories,Curtains,Elegance Polyester Multicolor Abstract Eyelet...,None,None,None
1,7b72c92c2f6c40268628ec5f14c6d590,Sathiyas Cotton Bath Towel,Baby Care >> Baby Bath & Skin >> Baby Bath Tow...,Specifications of Sathiyas Cotton Bath Towel (...,"{""product_specification""=>[{""key""=>""Machine Wa...",Baby Care,Baby Bath & Skin,Baby Bath Towels,Sathiyas Baby Bath Towels,"Sathiyas Cotton Bath Towel (3 Bath Towel, Red...",None,None


In [24]:
df_nlp["cat1"].unique()

array(['Home Furnishing ', 'Baby Care ', 'Watches ',
       'Home Decor & Festive Needs ', 'Kitchen & Dining ',
       'Beauty and Personal Care ', 'Computers '], dtype=object)

In [25]:
df_nlp["cat1"].value_counts()

Home Furnishing                150
Baby Care                      150
Watches                        150
Home Decor & Festive Needs     150
Kitchen & Dining               150
Beauty and Personal Care       150
Computers                      150
Name: cat1, dtype: int64

In [26]:
df_nlp = df_nlp[["uniq_id", "cat1", "description"]]
df_nlp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   uniq_id      1050 non-null   object
 1   cat1         1050 non-null   object
 2   description  1050 non-null   object
dtypes: object(3)
memory usage: 24.7+ KB


In [27]:
df_nlp.head()

,uniq_id,cat1,description
0,55b85ea15a1536d46b7190ad6fff8ce7,Home Furnishing,Key Features of Elegance Polyester Multicolor ...
1,7b72c92c2f6c40268628ec5f14c6d590,Baby Care,Specifications of Sathiyas Cotton Bath Towel (...
2,64d5d4a258243731dc7bbb1eef49ad74,Baby Care,Key Features of Eurospa Cotton Terry Face Towe...
3,d4684dcdc759dd9cdf41504698d737d8,Home Furnishing,Key Features of SANTOSH ROYAL FASHION Cotton P...
4,6325b6870c54cd47be6ebfbffa620ec7,Home Furnishing,Key Features of Jaipur Print Cotton Floral Kin...


In [29]:
df_nlp.to_csv("data/df.csv", index=False)

In [30]:
# Train / test
X_train = df_nlp.groupby("cat1", group_keys=False).apply(lambda x: x.sample(130))
X_train["cat1"].value_counts()

Baby Care                      130
Beauty and Personal Care       130
Computers                      130
Home Decor & Festive Needs     130
Home Furnishing                130
Kitchen & Dining               130
Watches                        130
Name: cat1, dtype: int64

In [31]:
X_test = pd.concat([df_nlp, X_train]).drop_duplicates(keep=False)
X_test["cat1"].value_counts()

Home Furnishing                20
Watches                        20
Home Decor & Festive Needs     20
Kitchen & Dining               20
Computers                      20
Beauty and Personal Care       20
Baby Care                      20
Name: cat1, dtype: int64

In [53]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210 entries, 1 to 1045
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cat1         210 non-null    object
 1   description  210 non-null    object
 2   prediction   210 non-null    object
dtypes: object(3)
memory usage: 6.6+ KB


In [56]:
# Création d'un dictionnaire {cat1: [description1, descroption2...]}
data = dict()
for i in X_train['cat1'].unique().tolist():
    data_slice = X_train[X_train['cat1'] == i]
    data[i] = data_slice['description'].tolist()

In [31]:
import classy_classification

In [32]:
nlp = spacy.load("en_core_web_md")

In [78]:
if 'text_categorizer' not in nlp.pipe_names:
    nlp.add_pipe(
    "text_categorizer", 
    config={
        "data": data, 
        "model": "spacy"
            }
    )
else:
    textcat = nlp.get_pipe('text_categorizer')

In [80]:
nlp.analyze_pipes(pretty=True)


============================= Pipeline Overview =============================

#   Component          Assigns               Requires   Scores             Retokenizes
-   ----------------   -------------------   --------   ----------------   -----------
0   tok2vec            doc.tensor                                          False      
                                                                                      
1   tagger             token.tag                        tag_acc            False      
                                                                                      
2   parser             token.dep                        dep_uas            False      
                       token.head                       dep_las                       
                       token.is_sent_start              dep_las_per_type              
                       doc.sents                        sents_p                       
                                                  

{'summary': {'tok2vec': {'assigns': ['doc.tensor'],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'tagger': {'assigns': ['token.tag'],
   'requires': [],
   'scores': ['tag_acc'],
   'retokenizes': False},
  'parser': {'assigns': ['token.dep',
    'token.head',
    'token.is_sent_start',
    'doc.sents'],
   'requires': [],
   'scores': ['dep_uas',
    'dep_las',
    'dep_las_per_type',
    'sents_p',
    'sents_r',
    'sents_f'],
   'retokenizes': False},
  'attribute_ruler': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'lemmatizer': {'assigns': ['token.lemma'],
   'requires': [],
   'scores': ['lemma_acc'],
   'retokenizes': False},
  'ner': {'assigns': ['doc.ents', 'token.ent_iob', 'token.ent_type'],
   'requires': [],
   'scores': ['ents_f', 'ents_p', 'ents_r', 'ents_per_type'],
   'retokenizes': False},
  'text_categorizer': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False}},
 'problems': {'tok2vec': 

In [79]:
print(nlp("Double Bedsheet Royal Bedsheet Perfact for Wedding & Gifting")._.cats)

{'Baby Care ': 0.8619778774533123, 'Beauty and Personal Care ': 0.10625778617782206, 'Computers ': 0.007515475384235211, 'Home Decor & Festive Needs ': 0.008507215081600976, 'Home Furnishing ': 0.010206855932189264, 'Kitchen & Dining ': 0.0033096443341650494, 'Watches ': 0.002225145636674945}


In [58]:
def cat_predict(description):
    result = nlp(description)._.cats
    return max(result, key=result.get)

In [59]:
X_test["prediction"] = X_test["description"].apply(lambda x: cat_predict(x))

In [60]:
X_test.iloc[30:40]

,cat1,description,prediction
275,Baby Care,Key Features of Babeezworld Baby Boy's Romper ...,Baby Care
292,Baby Care,Key Features of Toddla Baby Girl's Gathered Dr...,Baby Care
296,Computers,Specifications of I Ball IBCTDC 160/2gb/DDR2 w...,Computers
307,Home Decor & Festive Needs,Buy Aapno Rajasthan Marvel In Marble - Gold Em...,Home Decor & Festive Needs
313,Beauty and Personal Care,Flipkart.com: Buy Adidas Ice Drive and Dynamic...,Beauty and Personal Care
317,Beauty and Personal Care,Flipkart.com: Buy Archies VJ-552 Combo Set onl...,Beauty and Personal Care
318,Computers,Buy Asus Wireless AC2400 Dual-band Gigabit Rou...,Computers
321,Computers,Buy Apple MD199HN/A only for Rs. 5900 from Fli...,Computers
329,Computers,Buy Asus RT-AC68U Dual-band Wireless-AC1900 Gi...,Computers
332,Computers,Buy TRENDnet N150 Wireless ADSL 2 Modem Router...,Computers


In [61]:
import numpy as np
X_test["check"] = np.where(X_test["cat1"] == X_test["prediction"], 1, 0)

In [62]:
X_test["check"].value_counts()

1    136
0      4
Name: check, dtype: int64

In [1]:
# Accuracy
136/140

0.9714285714285714

---